In [ ]:
# Import packages

import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
from keras import models
from keras import layers
from keras import optimizers
from PIL import Image
import sklearn
from sklearn.model_selection import train_test_split
from statistics import mean
import unidecode


In [ ]:
# Function to process and arrange image data
def arrange_data(df):
    
    image_data = []
    img_paths = np.asarray(df.iloc[:, 0]) #First column is the image paths
    
    for i in tqdm(range(len(img_paths))):
    
            
        img = tf.keras.utils.load_img(img_paths[i],target_size=(200, 150, 3))
        img = tf.keras.utils.img_to_array(img)
        img = img/255
        image_data.append(img)
        
        
    X = np.array(image_data)
    Y = np.array(df.iloc[:,1:7])
    
    print("Shape of images:", X.shape)
    print("Shape of labels:", Y.shape)
    
    return X, Y


train_df = pd.read_csv('INSERT_FILEPATH_HERE/trainingData.csv')

# Modifying image path column
train_df['Image Path'] = train_df['Image Path'].str.replace('INSERT_FILEPATH_HERE','')
train_df['Image Path'] = train_df['Image Path'].str.replace('/images','')

X_train, Y_train = arrange_data(train_df)

test_df = pd.read_csv('testingData.csv')

test_df['Image Path'] = test_df['Image Path'].str.replace('INSERT_FILEPATH_HERE','')
test_df['Image Path'] = test_df['Image Path'].str.replace('/images','')

X_test, Y_test = arrange_data(test_df)

In [ ]:
num_classes = 6  # Number of genres

# Import architecture
from keras.applications import VGG16

# Load VGG model
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(200,150, 3))

# Freeze the layers except the last 4 layers
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False

# Create model
model = models.Sequential()

# Add the vgg convolutional base model
model.add(vgg_conv)

# Add layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='sigmoid'))

# Show a summary of the model. Check the number of trainable parameters
model.summary()

model.compile(optimizer=optimizers.RMSprop(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Construct training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,width_shift_range=0.2, height_shift_range=0.2, 
                         shear_range=0.15,horizontal_flip=True, fill_mode="nearest")

# Train the network
EPOCHS= 10
BS = 64

history = model.fit_generator(aug.flow(X_train, Y_train, batch_size=BS),validation_data=(X_test, Y_test), 
                    steps_per_epoch=len(X_train) // BS, epochs=EPOCHS)

In [4]:
# Save model for testing
model.save('Model_4d.h5')